# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symmer.utils import unit_n_sphere_cartesian_coords
from symmer.symplectic import PauliwordOp, QuantumState, array_to_QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [3]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp.from_list(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.000 |0> ->  0.000+1.000j |1>
 1.000 |1> ->  0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [6]:
psi = QuantumState([[0,0]])
had = PauliwordOp.from_list(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.000 |00>

After application of the Hadamard gate:

 0.500+0.000j |00> +
 0.500+0.000j |01> +
 0.500+0.000j |10> +
 0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [7]:
print(eq_superposition.state_op)

 0.500+0.000j ZZ +
 0.500+0.000j ZX +
 0.500+0.000j XZ +
 0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [8]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [9]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.345+0.112j |0001011110> +
 0.273+0.266j |1110111001> +
 0.049+0.063j |1000101011> +
 0.321+0.301j |1001101110> +
 0.242+0.063j |0110001111> +
 0.065+0.090j |1001111011> +
 0.119+0.318j |1001010000> +
 0.313+0.026j |1001000010> +
 0.332+0.007j |1111101000> +
 0.185+0.300j |0011111000>

 0.361+0.126j |1001010000> +
 0.274+0.056j |0000010111> +
 0.125+0.229j |1100010000> +
 0.154+0.239j |0110100101> +
 0.320+0.149j |1011011011> +
 0.059+0.044j |0001111001> +
 0.174+0.243j |1101011011> +
 0.369+0.318j |0110001110> +
 0.056+0.338j |0000111111> +
 0.180+0.140j |1111010100>


In [10]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.186+0.038j |0000010111> +
 0.038+0.230j |0000111111> +
 0.235+0.076j |0001011110> +
 0.040+0.030j |0001111001> +
 0.126+0.204j |0011111000> +
 0.251+0.216j |0110001110> +
 0.165+0.043j |0110001111> +
 0.104+0.162j |0110100101> +
 0.033+0.043j |1000101011> +
 0.213+0.018j |1001000010> +
 0.326+0.302j |1001010000> +
 0.218+0.205j |1001101110> +
 0.044+0.061j |1001111011> +
 0.217+0.102j |1011011011> +
 0.085+0.155j |1100010000> +
 0.118+0.165j |1101011011> +
 0.185+0.181j |1110111001> +
 0.122+0.095j |1111010100> +
 0.225+0.005j |1111101000>

Norm: (1+0j)


In [11]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

-1.799+0.000j YXZIZYXXZZ +
 0.721+0.000j YYIYYIZXZX +
 0.189+0.000j ZZZZXXZIZI +
 0.424+0.000j XZIIXYXIZZ +
-25.584+0.000j XZYXIYIXII +
 0.039+0.000j IXIIIXZXIY +
 2.361+0.000j YIZIZZZXXX +
 5.289+0.000j YZZYYXIYXZ +
-1.387+0.000j ZZYZXXYYYY +
 0.556+0.000j ZIIIZXIIIX

Expectation value <psi|op|psi> = (1.3579609771205203+2.220446049250313e-16j)


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [12]:
statevector = np.zeros(2**10)
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = array_to_QuantumState(statevector)
print(psi)
print(psi.conjugate * psi)

 0.408 |0000000000> +
 0.577 |0011111010> +
 0.707 |0111110100>
1.0


<4x4 sparse matrix of type '<class 'numpy.complex128'>'
	with 16 stored elements in Compressed Sparse Row format>

In [25]:
PauliwordOp.from_matrix(np.array([[1,1],[1,-1]])) * (1/np.sqrt(2)) * QuantumState([[0]])

 0.707+0.000j |0> +
 0.707+0.000j |1>